## Môi trường lập trình

In [ ]:
# Thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
import joblib            as jlb
from sklearn.decomposition   import PCA
from sklearn.preprocessing   import LabelEncoder, OneHotEncoder
from sklearn.preprocessing   import Binarizer, PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors       import KNeighborsClassifier
from sklearn                 import metrics
from sklearn.metrics         import accuracy_score, confusion_matrix
from sklearn.metrics         import auc,roc_curve
from sklearn.metrics         import precision_score, recall_score, f1_score
from sklearn.metrics         import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn                 import svm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from sklearn.model_selection import train_test_split

In [ ]:
# Bỏ qua các cảnh báo
np.seterr(divide = 'ignore')
warnings.filterwarnings('ignore')

## Tiền xử lý dữ liệu

In [ ]:
df = pd.read_csv('Hotel Reservations.csv')

df.head(5)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


### Thăm dò dữ liệu

Hiển thị thông tin bộ dữ liệu.

In [ ]:
print(f'Kích thước của bộ dữ liệu gồm {df.shape[0]} dòng và {df.shape[1]} cột.')

Kích thước của bộ dữ liệu gồm 36275 dòng và 19 cột.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [ ]:
# In số giá trị duy nhất của mỗi cột
df.nunique()

Booking_ID                              36275
no_of_adults                                5
no_of_children                              6
no_of_weekend_nights                        8
no_of_week_nights                          18
type_of_meal_plan                           4
required_car_parking_space                  2
room_type_reserved                          7
lead_time                                 352
arrival_year                                2
arrival_month                              12
arrival_date                               31
market_segment_type                         5
repeated_guest                              2
no_of_previous_cancellations                9
no_of_previous_bookings_not_canceled       59
avg_price_per_room                       3930
no_of_special_requests                      6
booking_status                              2
dtype: int64

In [ ]:
# In các giá trị duy nhất của các cột kiểu object
for column in df.select_dtypes(include=['object']).columns:
    unique_values = df[column].unique()
    print(f'Cột {column}:', unique_values)

Cột Booking_ID: ['INN00001' 'INN00002' 'INN00003' ... 'INN36273' 'INN36274' 'INN36275']
Cột type_of_meal_plan: ['Meal Plan 1' 'Not Selected' 'Meal Plan 2' 'Meal Plan 3']
Cột room_type_reserved: ['Room_Type 1' 'Room_Type 4' 'Room_Type 2' 'Room_Type 6' 'Room_Type 5'
 'Room_Type 7' 'Room_Type 3']
Cột market_segment_type: ['Offline' 'Online' 'Corporate' 'Aviation' 'Complementary']
Cột booking_status: ['Not_Canceled' 'Canceled']


In [ ]:
# In các giá trị duy nhất của các cột kiểu number
for column in df.select_dtypes(include=['number']).columns:
    unique_values = df[column].unique()
    print(f'Cột {column}:', unique_values)

Cột no_of_adults: [2 1 3 0 4]
Cột no_of_children: [ 0  2  1  3 10  9]
Cột no_of_weekend_nights: [1 2 0 4 3 6 5 7]
Cột no_of_week_nights: [ 2  3  1  4  5  0 10  6 11  7 15  9 13  8 14 12 17 16]
Cột required_car_parking_space: [0 1]
Cột lead_time: [224   5   1 211  48 346  34  83 121  44   0  35  30  95  47 256  99  12
 122   2  37 130  60  56   3 107  72  23 289 247 186  64  96  41  55 146
  32  57   7 124 169   6  51  13 100 139 117  39  86  19 192 179  26  74
 143 177  18 267 155  46 128  20  40 196 188  17 110  68  73  92 171 134
 320 118 189  16  24   8  10 182 116 123 105 443 317 286 148  14  85  25
  28  80  11 162  82  27 245 266 112  88  69 273   4  97  31  62 197 280
 185 160 104  22 292 109 126 303  81  54  15 161 147  87 127 418 156  58
 433 111 195 119  59  78 335 103  70  76 144  49  77  36  79  21  33 164
 152  43 102  71 209  93  53 302 239  45 167 113  84   9 166 174  61 151
  52  67 282  38 175  89 133  65  66  50 159 386 115 237 125  91  29 221
 213 198  75 180 236 120

In [ ]:
# Loại bỏ cột Booking_ID
df = df.drop(columns = ['Booking_ID'])

In [ ]:
# Chuyển kiểu cột dữ liệu
df['repeated_guest'] = df['repeated_guest'].astype(object)
df['required_car_parking_space'] = df['required_car_parking_space'].astype(object)

In [ ]:
# Đặt cột class là target
target = df['booking_status']
df = df.drop(columns = 'booking_status')

In [ ]:
# In số giá trị duy nhất của cột target
tar_unique_values = target.unique()
tar_unique_values

array(['Not_Canceled', 'Canceled'], dtype=object)

### Làm sạch dữ liệu

Kiểm tra giá trị thiếu.

In [ ]:
df.isnull().sum()

no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
dtype: int64

Kiểm tra giá trị nhiễu.

In [ ]:
# Sử dụng hàm để kiểm tra outlier cho tất cả các cột trong DataFrame
def check_outliers_using_3_sigma(df):
    has_outliers = True
    while has_outliers:
        has_outliers = False
        for col in df.columns:
            if df[col].dtype in ['int64', 'float64']:
                lech3sigma_tren = round(df[col].mean() + 3 * df[col].std(ddof=1))
                lech3sigma_duoi = round(df[col].mean() - 3 * df[col].std(ddof=1))
                df_filter = df[(df[col] > lech3sigma_tren) | (df[col] < lech3sigma_duoi)]

                if df_filter[col].count() > 0:
                    print(f'Cột {col} có {df_filter[col].count()} giá trị nhiễu là các giá trị ngoài khoảng ({lech3sigma_duoi}, {lech3sigma_tren})')
                    # Thay thế nhiễu bằng trung vị
                    trungvi = df[col].median()
                    df.loc[(df[col] > lech3sigma_tren) | (df[col] < lech3sigma_duoi), col] = trungvi
                    has_outliers = True
                else:
                    print(f'Cột {col}: Không có outlier.')

# Gọi hàm trên DataFrame df
check_outliers_using_3_sigma(df)

Cột no_of_adults có 16 giá trị nhiễu là các giá trị ngoài khoảng (0, 3)
Cột no_of_children có 1080 giá trị nhiễu là các giá trị ngoài khoảng (-1, 1)
Cột no_of_weekend_nights có 184 giá trị nhiễu là các giá trị ngoài khoảng (-2, 3)
Cột no_of_week_nights có 324 giá trị nhiễu là các giá trị ngoài khoảng (-2, 6)
Cột lead_time có 376 giá trị nhiễu là các giá trị ngoài khoảng (-173, 343)
Cột arrival_year: Không có outlier.
Cột arrival_month: Không có outlier.
Cột arrival_date: Không có outlier.
Cột no_of_previous_cancellations có 140 giá trị nhiễu là các giá trị ngoài khoảng (-1, 1)
Cột no_of_previous_bookings_not_canceled có 267 giá trị nhiễu là các giá trị ngoài khoảng (-5, 5)
Cột avg_price_per_room có 353 giá trị nhiễu là các giá trị ngoài khoảng (-2, 209)
Cột no_of_special_requests có 86 giá trị nhiễu là các giá trị ngoài khoảng (-2, 3)
Cột no_of_adults: Không có outlier.
Cột no_of_children: Không có outlier.
Cột no_of_weekend_nights: Không có outlier.
Cột no_of_week_nights: Không có out

Rời rạc hóa dữ liệu là các biến liên tục.

In [ ]:
# Rời rạc các biến lead_time và avg_price_per_room
df['lead_time_group'] = pd.qcut(df['lead_time'], 4)
df['avg_price_per_room_group'] = pd.qcut(df['avg_price_per_room'], 4)
df = df.drop(columns = ['lead_time', 'avg_price_per_room'])

In [ ]:
df = df.astype(object)

In [ ]:
columns_to_exclude = ['type_of_meal_plan', 'room_type_reserved', 'market_segment_type']

# Chuyển giá trị ở các ô thành dạng "tên cột (giá trị ở ô tương ứng)" cho các cột trừ các cột có giá trị số rời rạc và nhị phân
df = df.apply(lambda x: x.astype(str).map(lambda y: f'{x.name} ({y})') if x.name not in columns_to_exclude else x)

In [ ]:
df.shape

(36275, 17)

In [ ]:
df.head(5)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,lead_time_group,avg_price_per_room_group
0,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (2),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2017),arrival_month (10),arrival_date (2),Offline,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((22.999, 81.08])"
1,no_of_adults (2),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (3),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (11),arrival_date (6),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (1),"lead_time_group ((-0.001, 17.0])","avg_price_per_room_group ((99.45, 118.96])"
2,no_of_adults (1),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (1),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (2),arrival_date (28),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((-0.001, 17.0])","avg_price_per_room_group ((22.999, 81.08])"
3,no_of_adults (2),no_of_children (0),no_of_weekend_nights (0),no_of_week_nights (2),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (5),arrival_date (20),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((99.45, 118.96])"
4,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (1),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (4),arrival_date (11),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((17.0, 57.0])","avg_price_per_room_group ((81.08, 99.45])"


In [ ]:
df.dtypes

no_of_adults                            object
no_of_children                          object
no_of_weekend_nights                    object
no_of_week_nights                       object
type_of_meal_plan                       object
required_car_parking_space              object
room_type_reserved                      object
arrival_year                            object
arrival_month                           object
arrival_date                            object
market_segment_type                     object
repeated_guest                          object
no_of_previous_cancellations            object
no_of_previous_bookings_not_canceled    object
no_of_special_requests                  object
lead_time_group                         object
avg_price_per_room_group                object
dtype: object

## Xây dựng bài toán

In [ ]:
df.shape

(36275, 17)

In [ ]:
target.shape

(36275,)

In [ ]:
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,lead_time_group,avg_price_per_room_group
0,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (2),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2017),arrival_month (10),arrival_date (2),Offline,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((22.999, 81.08])"
1,no_of_adults (2),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (3),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (11),arrival_date (6),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (1),"lead_time_group ((-0.001, 17.0])","avg_price_per_room_group ((99.45, 118.96])"
2,no_of_adults (1),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (1),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (2),arrival_date (28),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((-0.001, 17.0])","avg_price_per_room_group ((22.999, 81.08])"
3,no_of_adults (2),no_of_children (0),no_of_weekend_nights (0),no_of_week_nights (2),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (5),arrival_date (20),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((99.45, 118.96])"
4,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (1),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (4),arrival_date (11),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((17.0, 57.0])","avg_price_per_room_group ((81.08, 99.45])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,no_of_adults (3),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (6),Meal Plan 1,required_car_parking_space (0),Room_Type 4,arrival_year (2018),arrival_month (8),arrival_date (3),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (1),"lead_time_group ((57.0, 118.0])","avg_price_per_room_group ((118.96, 184.0])"
36271,no_of_adults (2),no_of_children (0),no_of_weekend_nights (1),no_of_week_nights (3),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (10),arrival_date (17),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (2),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((81.08, 99.45])"
36272,no_of_adults (2),no_of_children (0),no_of_weekend_nights (2),no_of_week_nights (6),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (7),arrival_date (1),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (2),"lead_time_group ((118.0, 309.0])","avg_price_per_room_group ((81.08, 99.45])"
36273,no_of_adults (2),no_of_children (0),no_of_weekend_nights (0),no_of_week_nights (3),Not Selected,required_car_parking_space (0),Room_Type 1,arrival_year (2018),arrival_month (4),arrival_date (21),Online,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),no_of_special_requests (0),"lead_time_group ((57.0, 118.0]

### Chuẩn bị dữ liệu

In [ ]:
def onehot(col, df):
  enc = OneHotEncoder(sparse = False) # dạng
  df_enc = pd.DataFrame(enc.fit_transform(df[[col]]), columns = df[col].unique())
  df.drop(columns = col, inplace = True)
  df_combined = pd.concat([df, df_enc], axis=1)
  return df_combined

In [ ]:
for i in df.select_dtypes(include = ['object']).columns:
  df = onehot(i, df)

# Tạo một bản sao của dataframe này để thuận tiện cho việc đánh giá mô hình
data = df.copy()

df = df.replace({1: True, 0: False})

df

,no_of_adults (2),no_of_adults (1),no_of_adults (3),no_of_adults (0),no_of_children (0),no_of_children (1),no_of_weekend_nights (1),no_of_weekend_nights (2),no_of_weekend_nights (0),no_of_weekend_nights (3),...,no_of_special_requests (3),no_of_special_requests (2),"lead_time_group ((118.0, 309.0])","lead_time_group ((-0.001, 17.0])","lead_time_group ((17.0, 57.0])","lead_time_group ((57.0, 118.0])","avg_price_per_room_group ((22.999, 81.08])","avg_price_per_room_group ((99.45, 118.96])","avg_price_per_room_group ((81.08, 99.45])","avg_price_per_room_group ((118.96, 184.0])"
0,False,False,True,False,True,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,False
1,False,False,True,False,True,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,True
2,False,True,False,False,True,False,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
3,False,False,True,False,True,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
4,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,False,False,False,True,True,False,False,False,True,False,...,False,False,False,False,False,True,True,False,False,False
36271,False,False,True,False,True,False,False,True,False,False,...,True,False,False,True,False,False,False,False,True,False
36272,False,False,True,False,True,False,False,False,True,False,...,True,False,False,True,False,False,False,False,True,False
36273,False,False,True,False,True,False,True,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [ ]:
# Trở về dạng transaction (nếu cần)
item = []
for index, row in df.iterrows():
  phantu = []
  for column, value in row.iteritems():
    if value == True:
      phantu.append(column)
  item.append(phantu)

df_item = pd.DataFrame({'Items' : item})
df_return = pd.concat([df_item, target], axis=1)
df_return

,Items,booking_status
0,"[no_of_adults (3), no_of_children (0), no_of_w...",Not_Canceled
1,"[no_of_adults (3), no_of_children (0), no_of_w...",Not_Canceled
2,"[no_of_adults (1), no_of_children (0), no_of_w...",Canceled
3,"[no_of_adults (3), no_of_children (0), no_of_w...",Canceled
4,"[no_of_adults (3), no_of_children (0), no_of_w...",Canceled
...,...,...
36270,"[no_of_adults (0), no_of_children (0), no_of_w...",Not_Canceled
36271,"[no_of_adults (3), no_of_children (0), no_of_w...",Canceled
36272,"[no_of_adults (3), no_of_children (0), no_of_w...",Not_Canceled
36273,"[no_of_adults (3), no_of_children (0), no_of_w...",Canceled


### Khai thác mẫu phổ biến

In [ ]:
frequent_patterns = fpgrowth(df, min_support = 0.5, use_colnames=True)
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].apply(lambda x: ', '.join(x))

# Sắp xếp chỉ số dựa trên số lượng từ
word_counts = frequent_patterns['itemsets'].str.split(",").apply(len)
sorted_indices = np.argsort(word_counts)

# Sắp xếp lại cột "Itemsets" dựa trên chỉ số đã sắp xếp
frequent_patterns['itemsets'] = frequent_patterns['itemsets'].iloc[sorted_indices].values

# Lưu các tập phổ biến lớn nhất
frequent_patterns_filter = frequent_patterns.copy()
frequent_patterns_reverse = frequent_patterns.iloc[::-1]
for valuehigh in frequent_patterns_reverse['itemsets']:
  for valuelow in frequent_patterns['itemsets']:
    dem = 0
    if len(valuelow.split(', ')) == len(valuehigh.split(', ')):
      break
    for l in valuelow.split(', '):
      if l in valuehigh.split(', '):
        dem = dem + 1
    if dem == len(valuelow.split(', ')):
      frequent_patterns_filter = frequent_patterns_filter[frequent_patterns['itemsets'] != valuelow]

In [ ]:
frequent_patterns

,support,itemsets
0,1.000000,no_of_previous_bookings_not_canceled (0)
1,1.000000,Complementary
2,0.974363,arrival_year (2018)
3,0.969014,no_of_adults (3)
4,0.955396,Meal Plan 1
...,...,...
422,0.502329,"required_car_parking_space (0), Meal Plan 1, n..."
423,0.504121,"required_car_parking_space (0), no_of_previous..."
424,0.502329,"required_car_parking_space (0), no_of_previous..."
425,0.502329,"required_car_parking_space (0), no_of_previous..."


### Chọn lọc mẫu

In [ ]:
frequent_patterns_filter

,support,itemsets
410,0.537092,"required_car_parking_space (0), no_of_previous..."
411,0.537092,"required_car_parking_space (0), Meal Plan 1, n..."
417,0.520276,"Meal Plan 1, no_of_previous_bookings_not_cance..."
418,0.520276,"required_car_parking_space (0), Meal Plan 1, n..."
421,0.504121,"required_car_parking_space (0), Meal Plan 1, n..."
422,0.502329,"required_car_parking_space (0), Meal Plan 1, n..."
423,0.504121,"required_car_parking_space (0), no_of_previous..."
424,0.502329,"required_car_parking_space (0), no_of_previous..."
425,0.502329,"required_car_parking_space (0), no_of_previous..."
426,0.502329,"required_car_parking_space (0), no_of_previous..."


### Chuyển đổi mẫu phổ biến thành dạng đặc trưng

In [ ]:
# Lấy các cột là mẫu phổ biến
df_filtered = df[df.columns[df.columns.isin(frequent_patterns.itemsets)]]

## XOR của các tập phổ biến
for item in frequent_patterns_filter['itemsets']:
  row = []
  value = []
  for index in item.split(', '):
    row.append(index)
  name = ' x '.join(row)

  for i in row:
    value.append(df_filtered[i])

  result = value[0]
  for i in range(1, len(value)):
    result = [x ^ y for x, y in zip(result, value[i])]

  df_filtered[name] = result

# Xây dựng đặc trưng
features = df.astype(int).values

In [ ]:
train = pd.concat([df_filtered, target], axis=1)
train

,no_of_adults (3),no_of_children (0),Meal Plan 1,required_car_parking_space (0),Room_Type 1,arrival_year (2018),Complementary,repeated_guest (0),no_of_previous_cancellations (0),no_of_previous_bookings_not_canceled (0),...,required_car_parking_space (0) x Meal Plan 1 x no_of_previous_bookings_not_canceled (0) x repeated_guest (0) x no_of_adults (3) x no_of_previous_cancellations (0),Meal Plan 1 x no_of_previous_bookings_not_canceled (0) x Room_Type 1 x repeated_guest (0) x no_of_previous_cancellations (0) x no_of_children (0),required_car_parking_space (0) x Meal Plan 1 x no_of_previous_bookings_not_canceled (0) x Room_Type 1 x no_of_previous_cancellations (0) x no_of_children (0),required_car_parking_space (0) x Meal Plan 1 x no_of_previous_bookings_not_canceled (0) x repeated_guest (0) x no_of_previous_cancellations (0) x Room_Type 1,required_car_parking_space (0) x Meal Plan 1 x no_of_previous_bookings_not_canceled (0) x repeated_guest (0) x arrival_year (2018) x no_of_previous_cancellations (0) x no_of_children (0),required_car_parking_space (0) x no_of_previous_bookings_not_canceled (0) x repeated_guest (0) x arrival_year (2018) x no_of_adults (3) x no_of_previous_cancellations (0) x no_of_children (0),required_car_parking_space (0) x no_of_previous_bookings_not_canceled (0) x Room_Type 1 x repeated_guest (0) x no_of_adults (3) x no_of_previous_cancellations (0) x no_of_children (0),required_car_parking_space (0) x no_of_previous_bookings_not_canceled (0) x Room_Type 1 x repeated_guest (0) x arrival_year (2018) x no_of_previous_cancellations (0) x no_of_children (0),required_car_parking_space (0) x no_of_previous_bookings_not_canceled (0) x repeated_guest (0) x arrival_year (2018) x no_of_previous_cancellations (0) x Complementary x no_of_children (0),booking_status
0,True,True,True,True,True,False,False,True,True,True,...,False,False,False,False,False,False,True,False,True,Not_Canceled
1,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,True,Not_Canceled
2,False,True,True,True,True,True,True,True,True,True,...,True,False,False,False,True,False,False,True,True,Canceled
3,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,True,True,True,True,True,Canceled
4,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,True,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,False,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,False,True,False,True,Not_Canceled
36271,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,True,True,True,True,True,Canceled
36272,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,True,True,True,True,True,Not_Canceled
36273,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,True,Canceled


### Xây dựng mô hình phân lớp

In [ ]:
y = train['booking_status']
X = train.drop(columns = 'booking_status')

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

#### Tập dữ liệu sử dụng mẫu thường xuyên

Huấn luyện mô hình

In [ ]:
##------------------------------------------------------------------------------
## Xây dựng mô hình
##------------------------------------------------------------------------------
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

SVC(kernel='linear')

Dự đoán kết quả

In [ ]:
# Dự đoán nhãn lớp cho tập kiểm tra
y_pred = model.predict(X_test)

result_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
result_df

,Actual,Predicted
30392,Not_Canceled,Canceled
6685,Canceled,Not_Canceled
8369,Not_Canceled,Canceled
2055,Not_Canceled,Not_Canceled
10969,Canceled,Not_Canceled
...,...,...
32859,Canceled,Not_Canceled
35405,Not_Canceled,Not_Canceled
36098,Not_Canceled,Not_Canceled
34412,Not_Canceled,Not_Canceled


Đánh giá mô hình

In [ ]:
# Tính độ chính xác
accuracy = accuracy_score(y_test, y_pred)
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(accuracy,2))
print("Confusion maxtrix:\n", conf_matrix)

Accuracy: 0.74
Confusion maxtrix:
 [[1041 1307]
 [ 587 4320]]


#### Tập dữ liệu không sử dụng mẫu thường xuyên



In [ ]:
data

,no_of_adults (2),no_of_adults (1),no_of_adults (3),no_of_adults (0),no_of_children (0),no_of_children (1),no_of_weekend_nights (1),no_of_weekend_nights (2),no_of_weekend_nights (0),no_of_weekend_nights (3),...,no_of_special_requests (3),no_of_special_requests (2),"lead_time_group ((118.0, 309.0])","lead_time_group ((-0.001, 17.0])","lead_time_group ((17.0, 57.0])","lead_time_group ((57.0, 118.0])","avg_price_per_room_group ((22.999, 81.08])","avg_price_per_room_group ((99.45, 118.96])","avg_price_per_room_group ((81.08, 99.45])","avg_price_per_room_group ((118.96, 184.0])"
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
36271,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
36272,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
36273,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


Giảm chiều dữ liệu.

In [ ]:
pca = PCA().fit(data)
nb_features = df.shape[1] - 1

In [ ]:
var = 0.0
for k in range(1, nb_features + 1):
    pca = PCA(k)
    pca.fit(df)

    newVar = pca.explained_variance_ratio_.sum() * 100
    print('   * k = %2d' %k, ': phương sai tích lũy ~ %.2f%%' %newVar,
          '--> tăng ~ %.2f%%' %(newVar - var))
    var = newVar

   * k =  1 : phương sai tích lũy ~ 9.12% --> tăng ~ 9.12%
   * k =  2 : phương sai tích lũy ~ 15.05% --> tăng ~ 5.93%
   * k =  3 : phương sai tích lũy ~ 20.46% --> tăng ~ 5.42%
   * k =  4 : phương sai tích lũy ~ 25.30% --> tăng ~ 4.84%
   * k =  5 : phương sai tích lũy ~ 29.57% --> tăng ~ 4.27%
   * k =  6 : phương sai tích lũy ~ 33.59% --> tăng ~ 4.02%
   * k =  7 : phương sai tích lũy ~ 37.48% --> tăng ~ 3.89%
   * k =  8 : phương sai tích lũy ~ 41.15% --> tăng ~ 3.67%
   * k =  9 : phương sai tích lũy ~ 44.67% --> tăng ~ 3.52%
   * k = 10 : phương sai tích lũy ~ 48.06% --> tăng ~ 3.39%
   * k = 11 : phương sai tích lũy ~ 51.28% --> tăng ~ 3.22%
   * k = 12 : phương sai tích lũy ~ 54.36% --> tăng ~ 3.07%
   * k = 13 : phương sai tích lũy ~ 57.30% --> tăng ~ 2.95%
   * k = 14 : phương sai tích lũy ~ 59.95% --> tăng ~ 2.65%
   * k = 15 : phương sai tích lũy ~ 62.34% --> tăng ~ 2.38%
   * k = 16 : phương sai tích lũy ~ 64.69% --> tăng ~ 2.35%
   * k = 17 : phương sai tích lũy ~ 66.57

Chọn k = 5.

In [ ]:
data_norm = pd.DataFrame(StandardScaler().fit_transform(df)) # tự động loại cột Class

## Áp dụng PCA
pca = PCA(n_components = 5)
pca.fit(data_norm)

PCA(n_components=5)

In [ ]:
B = pca.transform(data_norm)
df_pca = pd.DataFrame(B, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
df_combined = pd.concat([df_pca, target], axis=1)
df_combined.head()

,PC1,PC2,PC3,PC4,PC5,booking_status
0,2.814426,-2.354187,-1.286693,1.401844,0.779170,Not_Canceled
1,-1.053608,-0.639358,3.100373,-0.921595,-1.296118,Not_Canceled
2,1.370333,2.149927,0.286114,-2.052423,0.165892,Canceled
3,-0.061699,-1.008078,-0.523814,-1.067436,0.366849,Canceled
4,-0.566008,-0.810371,2.694823,-1.909017,-1.545015,Canceled


In [ ]:
y = target
X = df_pca
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

Huấn luyện mô hình

In [ ]:
##------------------------------------------------------------------------------
## Xây dựng mô hình
##------------------------------------------------------------------------------
model = svm.SVC(kernel = 'linear')
model.fit(X_train, y_train)

SVC(kernel='linear')

Dự đoán kết quả

In [ ]:
# Dự đoán nhãn lớp cho tập kiểm tra
y_pred = model.predict(X_test)

result_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
result_df

,Actual,Predicted
30392,Not_Canceled,Not_Canceled
6685,Canceled,Not_Canceled
8369,Not_Canceled,Not_Canceled
2055,Not_Canceled,Not_Canceled
10969,Canceled,Not_Canceled
...,...,...
32859,Canceled,Not_Canceled
35405,Not_Canceled,Not_Canceled
36098,Not_Canceled,Not_Canceled
34412,Not_Canceled,Not_Canceled


Đánh giá mô hình

In [ ]:
# Tính độ chính xác
accuracy = accuracy_score(y_test, y_pred)
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(accuracy,2))
print("Confusion maxtrix:\n", conf_matrix)

Accuracy: 0.68
Confusion maxtrix:
 [[   0 2348]
 [   0 4907]]
